In [ ]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/' , force_remount=True)
import face_recognition
import numpy as np
import cv2
import pandas as pd
import os
from google.colab.patches import cv2_imshow
import os
import PIL
import io
from base64 import b64decode, b64encode
import time
from datetime import datetime
import xlrd
import pandas as pd
import time
from datetime import date
import csv
today = date.today().strftime('%Y-%m-%d %H:%M:%S')
os.chdir('/content/drive/MyDrive/attendance')
import Video_Capture
capture_number = 0

Mounted at /content/drive/


In [ ]:
a = Video_Capture
video_stream = a.Video_Capture().video_stream
video_frame = a.Video_Capture().video_frame
js_to_image = a.Video_Capture().js_to_image
bbox_to_bytes = a.Video_Capture().bbox_to_bytes

video_stream()
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0


while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    
    image_bytes = b64decode(js_reply["img"].split(',')[1])         
    #img = js_to_image(js_reply["img"])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)
    # create transparent overlay for bounding box
    bbox_array = np.zeros([600,600,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #face_location = face_recognition.face_locations(img)
    # get face region coordinates
    face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") 
    faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w+10,y+h+10),(255,0,0),2)

      
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
      # update bbox so next frame gets new overlay
    bbox = bbox_bytes
    filename =  "/content/drive/MyDrive/attendance/test/4.png" 
    cv2.imwrite(filename, img)
    capture_number += 1

<IPython.core.display.Javascript object>

In [ ]:
def read_train_images(folder , dim):
  X = []
  encode = []
  y = []
  Y = []
  n_samples=0
  none_image = 0
  for i in os.listdir(folder):
      #print(i) 
      img_file = cv2.imread( folder +'/' + i)
      #cv2_imshow(img_file)
      if img_file is not None:
          img_orginal=cv2.resize(img_file,dim)
          face_location = face_recognition.face_locations(img_orginal)
          encoded_img = face_recognition.face_encodings(img_orginal, face_location)
          #print(face_location)            
          X.append(img_orginal)
          encode.append(encoded_img)
          name = os.path.split(i)[-1]
          Y.append(name)
          n_samples+=1    
      else:
        none_image=+1
        print(f'there is {none_image} "not image" file in this folder')

  return X , Y, encode
dim = (600,600)
train_folder =  "/content/drive/MyDrive/attendance/train"
x , y , encoded_images= read_train_images(train_folder, dim)

In [ ]:
def read_test_image(test_folder, dim, encoded_images, y):
  Attendees_list = []
  k=0
  for iter in os.listdir(test_folder):
    test_img = cv2.imread(test_folder + iter)
    j = 0
    for i in encoded_images:
      test_img=cv2.resize(test_img,dim)
      face_location_test = face_recognition.face_locations(test_img)
      encoded_test = face_recognition.face_encodings(test_img, face_location_test)
      #print(len(encoded_test))
      if len(encoded_test) == 0:
        results = []
        
      else:
        results = face_recognition.compare_faces(np.array(i), np.array(encoded_test))
        print(results)
        if len(results) == 0:
          print('No face were detected')
        else:
          if results[0]:
            Attendees_list.append(y[j])
          else:
            pass
      j+=1
    k+=1   
  return Attendees_list, k
test_folder = "/content/drive/MyDrive/attendance/test/"
raw_list, K = read_test_image(test_folder, dim, encoded_images, y)


[False]
[False]
[False]
[False]
[True]


In [ ]:
lis_of_attendees = []
[lis_of_attendees.append(x) for x in raw_list if x not in lis_of_attendees]

absent_list = list(set(y).difference(lis_of_attendees))
Absents = []
for i in absent_list:
  name = i.split('.')[0]
  Absents.append(name)

Attendees = []
for i in lis_of_attendees:
  name = i.split('.')[0]
  Attendees.append(name)
print(f'Attendees students are :{Attendees}')
print(f'Absents students are :{Absents}')

if K>len(raw_list):
  strange_no = K-len(raw_list)
  print(f'There are {strange_no} strange person in the class. Be careful!!')
  os.chdir('/content/drive/MyDrive/attendance/')
from openpyxl import Workbook, load_workbook
workbook =  load_workbook(filename="ClassList.xlsx")

worksheet = workbook.active

Attendees students are :['Brad Pitt']
Absents students are :['Ritvij', 'Triza', 'Nasar', 'Amin']


In [ ]:
time_now = datetime.now()
HMS = [str(time_now.hour) + ':' + str(time_now.minute) + ":" + str(time_now.second)]
worksheet[f'A{capture_number}'] =  today.split(' ')[0]
worksheet[f'B{capture_number}'] = str(HMS)
C = f'C{capture_number}'
if 'Amin' in Attendees:
  worksheet[f'C{capture_number}'] = 'Present'
else:
  worksheet[C] =  "Absent"
if 'Nasar' in Attendees:
  worksheet[f'D{capture_number}']= 'Present'
else:
  worksheet[f'D{capture_number}'] = 'Absent'
if 'Ritvij' in Attendees:
  worksheet[f'E{capture_number}'] = 'Present'
else:
  worksheet[f'E{capture_number}'] =  'Absent'
if 'Triza' in Attendees:
  worksheet[f'F{capture_number}'] = 'Present'
else:
  worksheet[f'F{capture_number}'] = 'Absent'
workbook.close()
workbook.save('ClassList.xlsx')